In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import BooleanType, FloatType
from pyspark.sql.functions import trim, lower, upper, col, count, regexp_replace, max, min, when,avg, round, rand

spark = SparkSession.builder\
    .appName("CleanImmatriculation")\
    .enableHiveSupport()\
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

df_immat = spark.sql("SELECT * FROM immatriculations_ext")
df_client = spark.sql("SELECT * FROM clients")

#Renommage de la colonne "nom" en "modele"
df_immat = df_immat.withColumnRenamed("nom", "modele")

#Suppression de la 1ère ligne de la table clients
df_client = df_client.filter(df_client['immatriculation'] != 'immatriculation')

#Normalisation des casses
df_immat = df_immat.withColumn("marque", lower(trim(col("marque"))))
df_immat = df_immat.withColumn("modele", lower(trim(col("modele"))))
df_immat = df_immat.withColumn("immatriculation", upper(trim(col("immatriculation"))))

df_client = df_client.withColumn("situationfamiliale", lower(trim(col("situationfamiliale"))))
df_client = df_client.withColumn("immatriculation", upper(trim(col("immatriculation"))))

#Correction du symbole "�" dans la colonne "longueur"
df_immat = df_immat.withColumn("longueur", regexp_replace(col("longueur"), "�", "e"))
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "�", "e"))

#Mettre à null les valeurs inférieures à 18 ans.
df_client = df_client.withColumn("age", when(col("age") == -1, None).otherwise(col("age")))
df_client = df_client.withColumn("age", when(col("age") == 0, None).otherwise(col("age")))

#Calculer la médiane et remplacer les nulls par la médiane.
mediane = df_client.approxQuantile("age", [0.5], 0.01)[0]
df_client = df_client.withColumn("age", when(col("age").isNull(), mediane).otherwise(col("age")))

#Fusionner seul/seule en celibataire
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "seule", "celibataire"))
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "seul", "celibataire"))
df_client = df_client.withColumn("situationfamiliale", regexp_replace(col("situationfamiliale"), "divorcee", "divorce(e)"))

# Remplacer "n/d" par null dans la colonne situationfamiliale
df_client = df_client.withColumn(
    "situationfamiliale",
    when(col("situationfamiliale") == None, "n/d").otherwise(col("situationfamiliale"))
)
df_client = df_client.withColumn(
    "situationfamiliale",
    when(trim(col("situationfamiliale")) == "", "n/d").otherwise(col("situationfamiliale"))
)
# Remplacer "?" par "n/d" dans la colonne situationfamiliale
df_client = df_client.withColumn(
    "situationfamiliale",
    regexp_replace(col("situationfamiliale"), r"\?", "n/d")
)
#Supprimer les n/d car les proportions le permettent
df_client= df_client.filter(col("situationfamiliale") != "n/d")

# Remplacer les nulls de taux
mediane_taux = df_client.approxQuantile("taux", [0.5], 0.01)[0]
df_client = df_client.withColumn("taux", when(col("taux").isNull(), mediane).otherwise(col("taux")))

#Correction des valeurs hors domaine (Création d'une colonne "taux_eligible")
df_client = df_client.withColumn(
    "taux_eligible",
    when((col("taux") >= 544) & (col("taux") <= 74185), True).otherwise(False)
)

#Fusionner F/F�minin/Femme en F
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "Femme", "F"))
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "F�minin", "F"))
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "F", "F"))

#Fusionner H/M/Masculin/Hommme en H
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "Masculin", "M"))
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "Homme", "M"))
df_client = df_client.withColumn("sexe", regexp_replace(col("sexe"), "H", "M"))

#Remplacer les null/? par n/d
df_client = df_client.withColumn("sexe", when(col("sexe") == None, "n/d").otherwise(col("sexe")))
df_client = df_client.withColumn("sexe",when(trim(col("sexe")) == "", "n/d").otherwise(col("sexe")))
df_client = df_client.withColumn("sexe",when(trim(col("sexe")) == "N/D", "n/d").otherwise(col("sexe")))
df_client = df_client.withColumn("sexe",regexp_replace(col("sexe"), r"\?", "n/d"))

#Supprimer les n/d si les proportions le permettent
df_client= df_client.filter(col("sexe") != "n/d")

df_client = df_client.withColumn(
    "nbenfantacharge",
    when(
        (col("situationfamiliale") == "celibataire") & (col("nbenfantacharge").isNull() | (col("nbenfantacharge") == -1)),
        0
    ).otherwise(col("nbenfantacharge"))
)

df_client = df_client.withColumn(
    "nbenfantacharge",
    when(
        (col("situationfamiliale").isin("marie(e)", "divorce(e)", "en couple")) & 
        (col("nbenfantacharge").isNull() | (col("nbenfantacharge") == -1)),
        2
    ).otherwise(col("nbenfantacharge"))
)

df_nulls = df_client.filter(col("deuxiemevoiture").isNull())
# Ajouter une colonne aléatoire et assigner 'true' ou 'false' selon les proportions
df_nulls_replaced = df_nulls.withColumn(
    "deuxiemevoiture",
    when(rand() < 0.13, True).otherwise(False)
)
# Filtrer les lignes sans 'null' dans 'deuxiemevoiture'
df_non_nulls = df_client.filter(col("deuxiemevoiture").isNotNull())
# Combiner les deux DataFrames
df_client = df_non_nulls.union(df_nulls_replaced)

#Suppression des doublons
df_client= df_client.dropDuplicates(["immatriculation"])

#Changer le type de la colonne
df_immat = df_immat.withColumn("occasion", df_immat["occasion"].cast(BooleanType()))

#Fusion des tables
df_client_immat= df_client.join(df_immat, on= "immatriculation")

#Supression des doublons
df_client_immat= df_client_immat.dropDuplicates(["immatriculation"])

# Ajout de la colonne 'categorie' avec des critères précis et exhaustifs
df_client_immat = df_client_immat.withColumn(
    "categorie",
    when(
        (col("prix") >= 50000),  # Catégorie la plus prioritaire
        "luxe"
    )
    .when(
        (col("puissance") >= 200) & (col("prix") >= 40000),
        "sportive"
    )
    .when(
        (col("longueur").isin("longue", "tres longue")) & (col("nbplaces") >= 5) & (col("prix") >= 35000),
        "suv/crossover"
    )
    .when(
        (col("longueur").isin("moyenne", "longue")) & (col("nbplaces") >= 5) & (col("prix") < 35000),
        "familiale"
    )
    .when(
        (col("longueur") == "courte") & (col("puissance") >= 100) & (col("prix") >= 20000),
        "citadine standard"
    )
    .when(
        (col("longueur") == "courte") & (col("puissance") < 100) & (col("prix") < 20000),
        "citadine economique"
    )
    .otherwise(None)  # Evite une catégorie 'autre' non désirée
)



Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/20 10:23:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/20 10:23:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
#Compter les null
df_null_counts = df_client_immat.select(
    count(col("nbenfantacharge")).alias("non_null"),
    count(when(col("nbenfantacharge").isNull(), 1)).alias("null")
)

# Afficher le résultat
df_null_counts.show()

+--------+----+
|non_null|null|
+--------+----+
|   99401|   0|
+--------+----+



In [3]:
value_client = df_client_immat.groupBy("nbenfantacharge").count()
value_client.show()

+---------------+-----+
|nbenfantacharge|count|
+---------------+-----+
|              1|16425|
|              3|11451|
|              4| 9960|
|              2|16592|
|              0|44973|
+---------------+-----+

